# Setup

In [1]:
%pip install graphdatascience

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 8.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=379a72123d62d90ff62b7eed2b560ccfb308724fef36a6db5e7b5fe103f40c9b
  Stored in directory: /root/.cache/pip/wheels/cb/12/66/764554d079caad4b9a11a02cfc0d200dd876d12935b9cf7e64
Successfully built neo4j


In [ ]:
# Import the client
from graphdatascience import GraphDataScience

# Replace with the actual URI, username, and password
AURA_CONNECTION_URI = "neo4j+s://xxxxxxxx.databases.neo4j.io"
AURA_USERNAME = "neo4j"
AURA_PASSWORD = ""

# Configure the client with AuraDS-recommended settings
gds = GraphDataScience(
    AURA_CONNECTION_URI,
    auth=(AURA_USERNAME, AURA_PASSWORD),
    aura_ds=True
)

# Load data from Neo4j with native projections

In [ ]:
gds.run_cypher("""
    MERGE (a:EngineeringManagement {name: 'Alistair'})
    MERGE (j:EngineeringManagement {name: 'Jennifer'})
    MERGE (d:Developer {name: 'Leila'})
    MERGE (a)-[:MANAGES {start_date: 987654321}]->(d)
    MERGE (j)-[:MANAGES {start_date: 123456789, end_date: 987654321}]->(d)
""")

""


In [ ]:
# Project a graph using the shorthand syntax
shorthand_graph, result = gds.graph.project(
    "shorthand-example-graph",
    ["EngineeringManagement", "Developer"],
    ["MANAGES"]
)

print(result)

nodeProjection            {'EngineeringManagement': {'label': 'Engineeri...
relationshipProjection    {'MANAGES': {'orientation': 'NATURAL', 'aggreg...
graphName                                           shorthand-example-graph
nodeCount                                                                 3
relationshipCount                                                         2
projectMillis                                                             4
Name: 0, dtype: object


In [ ]:
# Project a graph using the extended syntax
extended_form_graph, result = gds.graph.project(
    "extended-form-example-graph",
    {
        "PersonEM": {
            "label": "EngineeringManagement"
        },
        "PersonD": {
            "label": "Developer"
        }
    },
    {
        "KNOWS": {
            "type": "MANAGES",
            "orientation": "UNDIRECTED",
            "properties": {
                "start_date": {
                    "property": "start_date"
                },
                "end_date": {
                    "property": "end_date",
                    "defaultValue": 999999999
                }
            }
        }
    }
)

print(result)

nodeProjection            {'PersonD': {'label': 'Developer', 'properties...
relationshipProjection    {'KNOWS': {'orientation': 'UNDIRECTED', 'aggre...
graphName                                       extended-form-example-graph
nodeCount                                                                 3
relationshipCount                                                         4
projectMillis                                                             5
Name: 0, dtype: object


# Use the Graph Catalog

In [ ]:
# List all in-memory graphs
all_graphs = gds.graph.list()

print(all_graphs)

                                  degreeDistribution  \
0  {'p99': 4, 'min': 2, 'max': 4, 'mean': 2.66666...   
1  {'p99': 1, 'min': 0, 'max': 1, 'mean': 0.66666...   

                     graphName database memoryUsage  sizeInBytes  nodeCount  \
0  extended-form-example-graph    neo4j    4354 KiB      4459352          3   
1      shorthand-example-graph    neo4j     257 KiB       263976          3   

   relationshipCount                                      configuration  \
0                  4  {'relationshipProjection': {'KNOWS': {'orienta...   
1                  2  {'relationshipProjection': {'MANAGES': {'orien...   

    density                         creationTime  \
0  0.666667  2022-03-29T13:25:47.786848000+00:00   
1  0.333333  2022-03-29T13:25:44.303603000+00:00   

                      modificationTime  \
0  2022-03-29T13:25:47.793019000+00:00   
1  2022-03-29T13:25:44.308548000+00:00   

                                              schema  
0  {'relationships': {'KNOWS

In [ ]:
# Check whether the "shorthand-example-graph" graph exists in memory
graph_exists = gds.graph.exists("shorthand-example-graph")

print(graph_exists)

graphName    shorthand-example-graph
exists                          True
Name: 0, dtype: object


In [ ]:
# Drop a graph object and keep the result of the call
result = gds.graph.drop(shorthand_graph)

# Print the result
print(result)

# Drop a graph object and just print the result of the call
gds.graph.drop(extended_form_graph)

graphName                                      shorthand-example-graph
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                            3
relationshipCount                                                    2
configuration        {'relationshipProjection': {'MANAGES': {'orien...
density                                                       0.333333
creationTime                       2022-03-29T13:25:44.303603000+00:00
modificationTime                   2022-03-29T13:25:44.308548000+00:00
schema               {'relationships': {'MANAGES': {}}, 'nodes': {'...
Name: 0, dtype: object


graphName                                  extended-form-example-graph
database                                                         neo4j
memoryUsage                                                           
sizeInBytes                                                         -1
nodeCount                                                            3
relationshipCount                                                    4
configuration        {'relationshipProjection': {'KNOWS': {'orienta...
density                                                       0.666667
creationTime                       2022-03-29T13:25:47.786848000+00:00
modificationTime                   2022-03-29T13:25:47.793019000+00:00
schema               {'relationships': {'KNOWS': {'end_date': 'Floa...
Name: 0, dtype: object

# Cleanup

In [ ]:
# Delete on-disk data
gds.run_cypher("""
    MATCH (example)
    WHERE example:EngineeringManagement OR example:Developer
    DETACH DELETE example
""")

""
